In [ ]:
# Step 1: Install Dependencies
!pip install transformers torch pandas scikit-learn

In [ ]:
# Step 2: Load Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# Replace with your dataset (e.g., IMDb reviews)
data = pd.read_csv("datasets/sentiment_data/reviews.csv")

In [ ]:
# Step 3: Preprocess Data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['text'], data['sentiment'], test_size=0.2
)

In [ ]:
# Step 4: Tokenize Data
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

In [ ]:
# Step 5: Create PyTorch Dataset
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
# Step 6: Load Pre-trained BERT Model
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Step 7: Set Up Training Arguments
training_args = TrainingArguments(
    output_dir="./sentiment_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir='./logs',
)

In [ ]:
# Step 8: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Step 9: Train the Model
trainer.train()

In [ ]:
# Step 10: Save the Trained Model
model.save_pretrained("backend/ml_models/fine_tuned_bert_sentiment")
tokenizer.save_pretrained("backend/ml_models/fine_tuned_bert_sentiment")